# Statistical Test Specification for TestSuites

In [ ]:
try:
    import evidently
except:
    !npm install -g yarn
    !pip install git+https://github.com/evidentlyai/evidently.git

In [ ]:
import pandas as pd
import numpy as np

from scipy.stats import mannwhitneyu
from sklearn import datasets, ensemble, model_selection

from evidently import ColumnMapping
from evidently.calculations.stattests import StatTest
from evidently.options import DataDriftOptions
from evidently.test_suite import TestSuite
from evidently.tests import *

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## Prepare Datasets

In [ ]:
#Dataset for Data Quality and Integrity
adult_data = datasets.fetch_openml(name='adult', version=2, as_frame='auto')
adult = adult_data.frame

adult_ref = adult[~adult.education.isin(['Some-college', 'HS-grad', 'Bachelors'])]
adult_cur = adult[adult.education.isin(['Some-college', 'HS-grad', 'Bachelors'])]

adult_cur.iloc[:2000, 3:5] = np.nan

## Data Drift Options

**Notes**: 
You can specify stattest for features and/or model output in DataDriftOptions

* all_features_stattest: Defines a custom statistical test for drift detection in the Data Drift report for all features
* num_features_stattest: Defines a custom statistical test for drift detection in the Data Drift report for numerical features only
* cat_features_stattest: Defines a custom statistical test for drift detection in the Data Drift report for categorical features only
* per_feature_stattest: Defines a custom statistical test for drift detection in the Data Drift report per feature

**Available stattests**:  
* 'ks' 
* 'z' 
* 'chisquare' 
* 'jensenshannon' 
* 'kl_div' 
* 'psi' 
* 'wasserstein'
* 'anderson'
* 'cramer_von_mises'
* 'g_test'

You can implement a custom drift test and use it in DataDriftOptions. Just define a function that takes two pd.Series (reference and current data) and returns a number (e.g. p_value or distance)

**Usage**:
- TestSuite(tests=[TestFeatureValueDrift(column_name='name', options=[options])])

## Setting the stattest for the whole dataset

In [ ]:
stat_test_option = DataDriftOptions(all_features_stattest='psi')

In [ ]:
data_drift_column_tests = TestSuite(tests=[
    TestFeatureValueDrift(column_name='education-num'),
    TestFeatureValueDrift(column_name='education-num', options=stat_test_option)
])

data_drift_column_tests.run(reference_data=adult_ref, current_data=adult_cur)
data_drift_column_tests

In [ ]:
data_drift_dataset_tests = TestSuite(tests=[
    TestShareOfDriftedColumns(options=stat_test_option),
])

data_drift_dataset_tests.run(reference_data=adult_ref, current_data=adult_cur)
data_drift_dataset_tests

## Setting the stattest for numerical and categorical features

In [ ]:
stat_test_option = DataDriftOptions(num_features_stattest='psi', cat_features_stattest='jensenshannon')

In [ ]:
data_drift_dataset_tests = TestSuite(tests=[
    TestShareOfDriftedColumns(options=stat_test_option),
])

data_drift_dataset_tests.run(reference_data=adult_ref, current_data=adult_cur)
data_drift_dataset_tests

## Setting the stattest for individual features

In [ ]:
from calendar import c


per_feature_stattest = {x: 'wasserstein' for x in ['age', 'education-num']}
for column in ['sex', 'class']:
    per_feature_stattest[column] = 'z'
for column in ['workclass', 'education']:
    per_feature_stattest[column] = 'kl_div'
for column in [ 'relationship', 'race',  'native-country']:
    per_feature_stattest[column] = 'jensenshannon'
for column in ['fnlwgt','hours-per-week']:
    per_feature_stattest[column] = 'anderson'
for column in ['capital-gain','capital-loss']:
    per_feature_stattest[column] = 'cramer_von_mises'
for column in ['occupation', 'marital-status']:
    per_feature_stattest[column] = 'g_test'

In [ ]:
per_feature_stattest

In [ ]:
stat_test_option = DataDriftOptions(per_feature_stattest=per_feature_stattest)

In [ ]:
data_drift_dataset_tests = TestSuite(tests=[
    TestShareOfDriftedColumns(options=stat_test_option),
])

data_drift_dataset_tests.run(reference_data=adult_ref, current_data=adult_cur)
data_drift_dataset_tests

## Custom Drift detection test

In [ ]:
def _mann_whitney_u(reference_data: pd.Series, current_data: pd.Series, _feature_type: str, threshold: float):
    p_value = mannwhitneyu(np.array(reference_data), np.array(current_data))[1]
    return p_value, p_value < threshold

mann_whitney_stat_test = StatTest(
    name="mann-whitney-u",
    display_name="mann-whitney-u test",
    func=_mann_whitney_u,
    allowed_feature_types=["num"]
)

In [ ]:
stat_test_option = DataDriftOptions(num_features_stattest=mann_whitney_stat_test)

In [ ]:
data_drift_dataset_tests = TestSuite(tests=[
    TestShareOfDriftedColumns(options=stat_test_option),
])

data_drift_dataset_tests.run(reference_data=adult_ref, current_data=adult_cur)
data_drift_dataset_tests